### import packages

In [ ]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity

### Specify directories to data files

In [ ]:
lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
bad_roi='/test/bad_roi_list.npy'
spot_dir = '/test/LHA3_R3_medium/spots/'
out_dir = '/test/LHA3_R3_medium/spots/'
img_dir='/test/LHA3_R3_medium/stitching/export.n5'

### Load segmentation mask

In [ ]:
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

### Extract ROI properties
The roi_prop function exports the following: 
- ROI ID
- position (x, y, z in um, pre-expansion)
- size (volume, pre-expansion) 
- mionr_axis_length
- major_axis_length
- eccentricity =  mionr_axis_length/major_axis_length
- solidity = ROI volume/ROI convex hull

In [ ]:
df=roi_prop.roi_prop(lb)

### Remove bad ROIs and save the file

In [ ]:
df_filtered=df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi=df_filtered.roi.astype(int)
df_filtered.to_csv(out_dir+'roi.csv')

### Lipofuscin subtraction (optional)

In [ ]:
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(spot_dir+'/'+'merged_points_c0.txt',delimiter=',')
channel_2=np.loadtxt(spot_dir+'/'+'merged_points_c1.txt',delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file

In [ ]:
spotcount=spot.spot_counts(lb,spot_dir)
spotcount.to_csv(out_dir+'/spots.csv')

### Intensity measurement and save file

In [ ]:
df_intensity=intensity.measure_intensity(lb, img_dir,'c0')
df_intensity.to_csv(out_dir+'/intennsity_c0.csv')

In [ ]:
df_intensity